In [30]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-mbm6mas_
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-mbm6mas_
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


In [3]:
import os
import torch
import numpy as np
from thingsvision import get_extractor
from thingsvision.utils.storing import save_features
from thingsvision.utils.data import ImageDataset, DataLoader
from torchvision import datasets, transforms
from PIL import Image
import torch.nn as nn
from typing import Any, Dict, List, Optional, Union

2 Physical GPUs, 2 Logical GPUs


2025-03-07 14:24:34.061472: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-07 14:24:38.048490: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22048 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:31:00.0, compute capability: 8.6
2025-03-07 14:24:38.049269: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 21594 MB memory:  -> device: 1, name: NVIDIA RTX A5000, pci bus id: 0000:ca:00.0, compute capability: 8.6


## Setting up the data directory

I am using the test set of CIFAR100 as an example of input stimuli. The data is stored in the `data` directory.

In [ ]:
# Set the paths
data_dir = '../data/'  # Use the already defined image_dir from cell 4
save_dir = '../data/cifar100_images'

# Only download the test set of CIFAR-100 dataset
test_dataset = datasets.CIFAR100(data_dir, train=False, download=True)

# Create directories for the test set
set_dir = os.path.join(save_dir, 'test')
os.makedirs(set_dir, exist_ok=True)

# Create directories for each class
for class_idx in range(len(test_dataset.classes)):
    class_dir = os.path.join(set_dir, test_dataset.classes[class_idx])
    os.makedirs(class_dir, exist_ok=True)

# Save each image from the test set
for i, (img, label) in enumerate(test_dataset):
    class_name = test_dataset.classes[label]
    img_path = os.path.join(set_dir, class_name, f'image_{i}.png')
    img.save(img_path)
    
    if i % 1000 == 0:
        print(f'Saved {i} images for test set')

print('Finished saving all CIFAR-100 test images')


Files already downloaded and verified
Saved 0 images for test set
Saved 1000 images for test set
Saved 2000 images for test set
Saved 3000 images for test set
Saved 4000 images for test set
Saved 5000 images for test set
Saved 6000 images for test set
Saved 7000 images for test set
Saved 8000 images for test set
Saved 9000 images for test set
Finished saving all CIFAR-100 test images


## Extract activations

In [4]:
#set your torch cache directory
os.environ['TORCH_HOME'] = '../cache'

#set your data directory where the images are stored
image_dir = '../data/cifar100_images/test'

#set the directory where the extracted features will be stored
output_dir = '../activations/cifar100'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [14]:
def extract_features(
    extractor: Any,
    module_name: str,
    image_path: str,
    out_path: str,
    batch_size: int,
    flatten_activations: bool,
    apply_center_crop: bool,
    class_names: Optional[List[str]]=None,
    file_names: Optional[List[str]]=None,
) -> Union[np.ndarray, torch.Tensor]:
    """Extract features for a single layer."""                                    
    dataset = ImageDataset(
        root=image_path,
        out_path=out_path,
        backend=extractor.get_backend(),
        transforms=extractor.get_transformations(apply_center_crop=apply_center_crop, resize_dim=256, crop_dim=224),
        class_names=class_names,
        file_names=file_names,
    )
    batches = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        backend=extractor.get_backend(),
        )
    features = extractor.extract_features(
        batches=batches,
        module_name=module_name,
        flatten_acts=flatten_activations,
        output_type="ndarray", # or "tensor" (only applicable to PyTorch models)
    )
    return features

def extract_all_layers(
    model_name: str,
    extractor: Any,
    image_path: str,
    out_path: str,
    batch_size: int,
    flatten_activations: bool,
    apply_center_crop: bool,
    layer: Any=nn.Linear,
    layer_names: Optional[List[str]]=None,
    file_format: str = "npy",
    class_names: Optional[List[str]]=None,
    file_names: Optional[List[str]]=None,
) -> Dict[str, Union[np.ndarray, torch.Tensor]]:
    """Extract features for all selected layers and save them to disk."""
    features_per_layer = {}
    
    # If specific layer names are provided, extract only those layers
    if layer_names:
        for module_name, module in extractor.model.named_modules():
            if module_name in layer_names:
                features = extract_features(
                    extractor=extractor,
                    module_name=module_name,
                    image_path=image_path,
                    out_path=out_path,
                    batch_size=batch_size,
                    flatten_activations=flatten_activations,
                    apply_center_crop=apply_center_crop,
                    class_names=class_names,
                    file_names=file_names,
                )
                features_per_layer[module_name] = features
                save_features(features, out_path=f'{out_path}/{model_name}/{module_name}', file_format=file_format)
    # Otherwise extract by layer type
    else:
        for l, (module_name, module) in enumerate(extractor.model.named_modules(), start=1):
            if isinstance(module, layer):
                # extract features for layer "module_name"
                features = extract_features(
                    extractor=extractor,
                    module_name=module_name,
                    image_path=image_path,
                    out_path=out_path,
                    batch_size=batch_size,
                    flatten_activations=flatten_activations,
                    apply_center_crop=apply_center_crop,
                    class_names=class_names,
                    file_names=file_names,
                )
                # if you want, replace for simplicity with e.g., [f'conv_{l:02d}'], [f'fc_{l:02d}'], or [f'layer_{l:02d}']
                features_per_layer[f'{module_name}'] = features
                # save features to disk
                save_features(features, out_path=f'{out_path}/{model_name}/{module_name}', file_format=file_format)
    return features_per_layer


In [ ]:
models = [{'model_name':'resnet50', 'source':'torchvision', 'model_parameters':None},
          {'model_name':'mocov2-rn50', 'source':'ssl', 'model_parameters':None},
          {'model_name':'clip', 'source':'custom', 'model_parameters':{'variant': 'ViT-B/32'}},
          {'model_name':'alexnet', 'source':'torchvision', 'model_parameters':None}]

for model in models:
    model_name = model['model_name']
    source = model['source']
    model_parameters = model.get('model_parameters', None)
    print(f"Model: {model_name}, Source: {source}")
    extractor = get_extractor(
        model_name=model_name,
        source=source,
        device=device,
        pretrained=True,
        model_parameters=model_parameters
    )
    print(extractor.show_model())
    print("=====================================================")

Model: resnet50, Source: torchvision
Using device:  cuda
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
   

In [20]:
output_dir = '../activations/cifar100' #set the directory where the extracted features will be stored
image_dir = '../data/cifar100_images/test' # set the directory where the input are stored
pretrained = True # use pretrained model weights
model_path = None # if pretrained = False (i.e., randomly initialized weights) set path to model weights
batch_size = 256 # use a power of two (this can be any size, depending on the number of images for which you aim to extract features)
apply_center_crop = True # center crop images (set to False, if you don't want to center-crop images)
flatten_activations = True # whether or not features (e.g., of Conv layers) should be flattened
class_names = None  # optional list of class names for class dataset
file_names = None # optional list of file names according to which features should be sorted
file_format = "npy" # format with which to save features to disk (can be set to "mat", "txt", "npy", "hdf5")
device = 'cuda' if torch.cuda.is_available() else 'cpu'


models = [{'model_name':'resnet50', 'source':'torchvision', 'model_parameters':None, 'layers_to_extract':['fc']},
          {'model_name':'mocov2-rn50', 'source':'ssl', 'model_parameters':None, 'layers_to_extract':['fc']},
          {'model_name':'clip', 'source':'custom', 'model_parameters':{'variant': 'ViT-B/32'}, 'layers_to_extract':['visual']},
          {'model_name':'alexnet', 'source':'torchvision', 'model_parameters':None, 'layers_to_extract':['classifier.4']}]

for model in models:
    model_name = model['model_name']
    source = model['source']
    model_parameters = model.get('model_parameters', None)
    layers_to_extract = model['layers_to_extract']
    print(f"Model: {model_name}, Source: {source}")
    extractor = get_extractor(
        model_name=model_name,
        source=source,
        device=device,
        pretrained=True,
        model_parameters=model_parameters
    )
    extract_all_layers(
        model_name=model_name,
        extractor=extractor,
        image_path=image_dir,
        out_path=output_dir,
        batch_size=batch_size,
        flatten_activations=True,
        apply_center_crop=False,
        layer_names=layers_to_extract,
        file_format="npy",
    ) 

Model: resnet50, Source: torchvision
Using device:  cuda

...Creating dataset.


Batch: 100%|██████████| 40/40 [02:32<00:00,  3.82s/it]


...Features successfully extracted for all 10000 images in the database.

Output directory did not exist. Creating directories to save features...

...Features successfully saved to disk.

Model: mocov2-rn50, Source: ssl
Using device:  cuda

...Creating dataset.


Batch: 100%|██████████| 40/40 [00:30<00:00,  1.31it/s]


...Features successfully extracted for all 10000 images in the database.

Output directory did not exist. Creating directories to save features...

...Features successfully saved to disk.

Model: clip, Source: custom
Using device:  cuda

...Creating dataset.


Batch: 100%|██████████| 40/40 [00:26<00:00,  1.49it/s]


...Features successfully extracted for all 10000 images in the database.

Output directory did not exist. Creating directories to save features...

...Features successfully saved to disk.

Model: alexnet, Source: torchvision
Using device:  cuda

...Creating dataset.


Batch: 100%|██████████| 40/40 [00:22<00:00,  1.78it/s]


...Features successfully extracted for all 10000 images in the database.

Output directory did not exist. Creating directories to save features...

...Features successfully saved to disk.

